# Data-Set Overview Plots

In [ ]:
using QAOA, Distributions, Interpolations
using HDF5, Printf
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")

using PyCall
np = pyimport("numpy")

PATH = "/home/ubuntu/Archives/"
PLOT_PATH = "/home/ubuntu/Archives/plots/SK_model/paper/";

In [ ]:
using Revise, SpinFluctuations

In [ ]:
Base.show(io::IO, f::Float64) = @printf(io, "%1.4f", f)

## Small-/Large-gap data comparison

In [ ]:
N = 9
# N = 11
# N = 13
# N = 15
# N = 17
# N = 19

patterns_dict = Dict(
    9 => r"random_SK_instance_N_9_seed_(\d+)\.h5",
    11 => r"random_SK_instance_N_11_seed_(\d+)\.h5",
    13 => r"random_SK_instance_N_13_seed_(\d+)\.h5",
    15 => r"random_SK_instance_N_15_seed_(\d+)\.h5",
    17 => r"random_SK_instance_N_17_seed_(\d+)\.h5",
    19 => r"random_SK_instance_N_19_seed_(\d+)\.h5"
);

### Loading

In [ ]:
npts = 2048
coarse_times = range(0, 1, npts + 1)

exact_times = range(0, 1, 33);

In [ ]:
subdir = "small_gaps"
# subdir = "large_gaps"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)
instance_names = readdir(folder_name)
filter!(x -> !occursin("results", x), instance_names)
filter!(x -> !occursin("undecided", x), instance_names)
filter!(x -> !occursin("frustrated", x), instance_names);

In [ ]:
# most_frustrated_spins = Dict(zip(h5read(folder_name * @sprintf("most_undecided_spins_N_%i.h5", N), @sprintf("T_final_%.0f_tol_1e%.0f/seeds", 32768., log10(1e-6))),
#                                       h5read(folder_name * @sprintf("most_undecided_spins_N_%i.h5", N), @sprintf("T_final_%.0f_tol_1e%.0f/spin_idxs", 32768., log10(1e-6)))));

In [ ]:
most_frustrated_spins_seeds = h5read(folder_name * @sprintf("most_frustrated_spins_N_%i.h5", N), @sprintf("T_final_%.0f_tol_1e%.0f/seeds", 32768., log10(1e-6)))
most_frustrated_spins_idxs = h5read(folder_name * @sprintf("most_frustrated_spins_N_%i.h5", N), @sprintf("T_final_%.0f_tol_1e%.0f/spin_idxs", 32768., log10(1e-6)));
most_frustrated_spins = Dict(zip(most_frustrated_spins_seeds, eachrow(most_frustrated_spins_idxs)));

In [ ]:
ordered_seeds = []

mean_fields = Dict()
mean_scaled_flucs = Dict()
most_frustrated_flucs = Dict()

all_Hs = Dict()
all_eigenvals = Dict()
all_eigenstates = Dict()

# for (k, instance_name) in enumerate(instance_names[310:319]) # instance "23320"
for (k, instance_name) in enumerate(instance_names[1:200])
    seed = match(patterns_dict[N], instance_name)[1]    
    print(k, ", ")

    # eigenvalues and -vectors
    λ = h5read(folder_name * instance_name, "exact_ARPACK_LM_eigvals")
    all_eigvecs = h5read(folder_name * instance_name, "exact_ARPACK_LM_lowest_eigvecs")

    J_mat = h5read(folder_name * instance_name, "J")
    mf_problem = Problem(0, J_mat)

    # mean-field solutions
    T_final = 32768.
    tol = 1e-6
    mf_sol = h5read(folder_name * "results_" * instance_name, @sprintf("mean_field_sol_T_final_%.0f_tol_1e%.0f", T_final, log10(tol)))
    sigma_star = sign.(mf_sol)
    h = mf_problem.local_fields
    J = mf_problem.couplings
    E_star = sum([-h[l] * sigma_star[l] for l in 1:N-1]) + sum([-J[i, j] * sigma_star[i] * sigma_star[j] for i in 1:N-1 for j in (i+1):N-1]) 
    
    # if isapprox(E_star, λ[1, end], atol=1e-5) || isapprox(E_star, λ[2, end], atol=1e-2)
    #     continue
    # end
        
    sol_t = h5read(folder_name * "results_" * instance_name, @sprintf("mean_field_T_final_%.0f_tol_1e%.0f/times", T_final, log10(tol)))
    sol_u = h5read(folder_name * "results_" * instance_name, @sprintf("mean_field_T_final_%.0f_tol_1e%.0f/trajectories", T_final, log10(tol)))

    n_vals(xyz) = reduce(hcat, [sol_u[k, xyz, :] for k in 1:size(sol_u)[1]])
    n_coarse(n_xyz) = reduce(hcat, [map(linear_interpolation(sol_t, n_xyz[spin_nr, :], extrapolation_bc=Line()), T_final .* coarse_times) for spin_nr in 1:N-1]) |> transpose
    nx_coarse = n_coarse(n_vals(1)) 
    ny_coarse = n_coarse(n_vals(2))
    nz_coarse = n_coarse(n_vals(3))
    mean_fields[seed] = [nx_coarse, ny_coarse, nz_coarse]

    # fluctuations
    T_final = 32000
    tol = 1e-8
    all_flucs = h5read(folder_name * "results_" * instance_name, @sprintf("fluctuations_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), npts))
    lyapunov_exponent = sum(all_flucs, dims=1)
    if sum(lyapunov_exponent) |> abs < 1e4 # discard non-converged ones

        # get "normal" trajectories 
        regular_trajectories = filter!(x -> x != most_frustrated_spins[seed][1], collect(1:N-1))
        # regular_trajectories = filter!(x -> x != most_frustrated_spins[seed][2], regular_trajectories)

        scale_factors = [1 .+ (nx_coarse[i, :] ./ (1 .+ sign(nz_coarse[i, end]) .* nz_coarse[i, :])).^2 for i in 1:N-1]
        # mean_scaled_flucs[seed] = mean([scale_factors[i] .* real(sqrt.(smoothen(all_flucs[i, :] .+ 0.0im, coarse_times))) for i in 1:N-1], dims=1)[1]
        mean_scaled_flucs[seed] = mean([scale_factors[i] .* real(sqrt.(all_flucs[i, :])) for i in regular_trajectories], dims=1)[1]

        most_frustrated_flucs[seed] = all_flucs[most_frustrated_spins[seed][1], :]
    end

    # adiabatic theorem...
    H_x = SpinFluctuations.hamiltonian(1, 0, mf_problem.local_fields, mf_problem.couplings)
    H_z = SpinFluctuations.hamiltonian(0, 1, mf_problem.local_fields, mf_problem.couplings)
    all_Hs[seed] = [H_x, H_z]

    eigenstate(n) = [all_eigvecs[k, :, n] for k in 1:length(exact_times)]
    num_eig_vecs = size(all_eigvecs)[3]
    all_eigenstates[seed] = [eigenstate(n) for n in 1:num_eig_vecs]
    all_eigenvals[seed] = [λ[n, :] for n in 1:num_eig_vecs]

    push!(ordered_seeds, seed)
end

In [ ]:
ordered_seeds |> size

## Data evaluation

In [ ]:
minigap_locs = Dict()
all_adiabatic_fracs = Dict()
all_frustrated_flucs = Dict()

seeds_to_max_fracs = Dict()

for seed in ordered_seeds
    try
        λs = all_eigenvals[seed]
        mingap = minimum(λs[2] .- λs[1])
        gap_idx = findfirst(x -> x == mingap, λs[2] .- λs[1]) 
        gaploc = exact_times[gap_idx]
        minigap_locs[seed] = gaploc

        nx_coarse, ny_coarse, nz_coarse = mean_fields[seed]

        top_idx = most_frustrated_spins[seed][1]
        scale_factor = 1 .+ (nx_coarse[top_idx, :] ./ (1 .+ sign(nz_coarse[top_idx, end]) .* nz_coarse[top_idx, :])).^2
        # all_frustrated_flucs[seed] = scale_factor .* (smoothen(most_frustrated_flucs[seed] .+ 0.0im, coarse_times) .|> sqrt |> real)
        all_frustrated_flucs[seed] = scale_factor .* (most_frustrated_flucs[seed] .|> sqrt |> real)

        H_x, H_z = all_Hs[seed]
        eigenstates = all_eigenstates[seed]

        overlap(n, H) = [eigenstates[n][k]' * H * eigenstates[1][k] for k in 1:length(exact_times)]
        frac_n(n) = abs.(overlap(n, H_z) .- overlap(n, H_x)) ./ (λs[n] .- λs[1])
        
        all_adiabatic_fracs[seed] = [frac_n(n) for n in 2:5]
        # all_adiabatic_fracs[seed] = [frac_n(n) for n in 2:4]
        
        # println(maximum(all_adiabatic_fracs[seed][1]), ", ", mean(all_adiabatic_fracs[seed][1]))
        seeds_to_max_fracs[seed] = maximum(all_adiabatic_fracs[seed][1])
    catch err
        println(seed)
    end
end

In [ ]:
seeds_and_max_fracs = sort([(k, v) for (k, v) in seeds_to_max_fracs], by=x->x[2]) |> reverse;

In [ ]:
seeds_and_max_fracs

## Plots

In [ ]:
mean_frac_1 = mean([seed_fracs[2][1] for seed_fracs in all_adiabatic_fracs])
mean_frac_2 = mean([seed_fracs[2][2] for seed_fracs in all_adiabatic_fracs])

# mean_frustrated_flucs = mean([(all_frustrated_flucs[seed] .- mean_scaled_flucs[seed]) ./ maximum(all_frustrated_flucs[seed] .- mean_scaled_flucs[seed]) for (seed, _) in all_adiabatic_fracs]);
# mean_frustrated_flucs = mean([(all_frustrated_flucs[seed] .- mean_scaled_flucs[seed]) for (seed, _) in all_adiabatic_fracs]);
mean_frustrated_flucs = mean([all_frustrated_flucs[seed] ./ maximum(all_frustrated_flucs[seed]) .- mean_scaled_flucs[seed] ./ maximum(mean_scaled_flucs[seed]) for (seed, _) in all_adiabatic_fracs]);

In [ ]:
num_instances = 100
num_instances = length(seeds_and_max_fracs)
mean_frac_1 = mean([all_adiabatic_fracs[seed][1] for (seed, _) in seeds_and_max_fracs[1:num_instances]])
mean_frac_2 = mean([all_adiabatic_fracs[seed][2] for (seed, _) in seeds_and_max_fracs[1:num_instances]])
mean_frac_3 = mean([all_adiabatic_fracs[seed][3] for (seed, _) in seeds_and_max_fracs[1:num_instances]])
mean_frac_4 = mean([all_adiabatic_fracs[seed][4] for (seed, _) in seeds_and_max_fracs[1:num_instances]])


mean_frustrated_flucs = mean([all_frustrated_flucs[seed] for (seed, _) in seeds_and_max_fracs[1:num_instances]]);
mean_mean_scaled_flucs = mean([mean_scaled_flucs[seed] for (seed, _) in seeds_and_max_fracs[1:num_instances]]);
mean_normalized_flucs = mean([(all_frustrated_flucs[seed] .- mean_scaled_flucs[seed]) ./ maximum(all_frustrated_flucs[seed] .- mean_scaled_flucs[seed]) for (seed, _) in seeds_and_max_fracs[1:num_instances]]);

In [ ]:
figure(figsize=(4, 4))
ax = subplot(211)
plot(exact_times, mean_frac_1, label="\$\\alpha = 1\$")
plot(exact_times, mean_frac_2, label="\$\\alpha = 2\$", lw=1.5)
plot(exact_times, mean_frac_3, label="\$\\alpha = 3\$", lw=2)
plot(exact_times, mean_frac_4, label="\$\\alpha = 4\$", lw=2.5)
plot(exact_times, mean_frac_1 .+ mean_frac_2 .+ mean_frac_3 .+ mean_frac_4, "-k", lw=2.5)
xlim(0., 1.)
ylim(0, )
ylabel("\$\\frac{|\\langle \\alpha|H_Z-H_X|0\\rangle|}{E_\\alpha - E_0}\$")
ax.set_xticklabels([])
legend(frameon=false)

ax = subplot(212)
plot(coarse_times,  mean_frustrated_flucs)
plot(coarse_times,  mean_frustrated_flucs .- mean_mean_scaled_flucs)
plot(coarse_times,  mean_normalized_flucs)
xlim(0., 1.)
ylim(0, )
xlabel("\$s\$")
tight_layout()

In [ ]:
figure(figsize=(4, 4))
ax = subplot(211)
semilogy(exact_times, mean_frac_1, "-ok", lw=1.5, ms=3)
xlim(0., 1.)
ylim(1e-1, 1e2)
ylabel("\$\\frac{|\\langle 1|H_Z-H_X|0\\rangle|}{E_1 - E_0}\$")
ax.set_xticklabels([])

ax = subplot(212)
plot(coarse_times,  mean_normalized_flucs)
xlim(0., 1.)
ylim(0, )
xlabel("\$s\$")
ylabel("\$\\mathrm{Re}\\;\\delta z_i(s)\$")
tight_layout()

### Shifted to center

In [ ]:
padded_exact_times = range(0, 1, 2(size(exact_times)[1] - 1) + 1)
padded_coarse_times = range(0, 1, 2(size(coarse_times)[1] - 1) + 1);

In [ ]:
num_instances = 100
num_instances = length(seeds_and_max_fracs)

mean_shifted_fracs = []
mean_shifted_flucs = []
for (seed, _) in seeds_and_max_fracs[1:num_instances]
    total_fracs = sum(all_adiabatic_fracs[seed])

    # get maximum from adiabatic fraction alongside index and time
    max_frac = maximum(total_fracs)
    max_frac_idx = findfirst(x -> x == max_frac, total_fracs)
    max_frac_time = exact_times[max_frac_idx]
    push!(mean_shifted_fracs, shift_idx_to_center(max_frac_idx, total_fracs))

    # find that also for coarse_times
    max_frac_idx_coarse = findfirst(x -> x == max_frac_time, coarse_times)
    push!(mean_shifted_flucs, shift_idx_to_center(max_frac_idx_coarse, all_frustrated_flucs[seed]))
end

mean_shifted_fracs = mean(mean_shifted_fracs)
mean_shifted_flucs = mean(mean_shifted_flucs);

In [ ]:
figure(figsize=(4, 4))
ax = subplot(211)
# plot(padded_exact_times, mean_shifted_fracs, "-ok", lw=1.5, ms=3)
semilogy(padded_exact_times, mean_shifted_fracs, "-ok", lw=1.5, ms=3)
xlim(0., 1.)
ylim(1e-1, 1e2)
ylabel("\$\\frac{|\\langle 1|H_Z-H_X|0\\rangle|}{E_1 - E_0}\$")
ax.set_xticklabels([])

ax = subplot(212)
plot(padded_coarse_times,  mean_shifted_flucs)
xlim(0., 1.)
ylim(0, )
xlabel("\$s\$")
ylabel("\$\\mathrm{Re}\\;\\delta z_i(s)\$")
tight_layout()

### Single-instance plots

In [ ]:
num_instances = 3

# for plot_seed in ordered_seeds
for (plot_seed, _) in seeds_and_max_fracs[1:num_instances]
    # plot_seed = ordered_seeds[20]
    print(plot_seed, "\t")
    frac_1, frac_2, frac_3, frac_4 = all_adiabatic_fracs[plot_seed]
    total_fracs = sum(all_adiabatic_fracs[plot_seed])

    # get maximum from adiabatic fraction alongside index and time
    max_frac = maximum(total_fracs)
    max_frac_idx = findfirst(x -> x == max_frac, total_fracs)
    max_frac_time = exact_times[max_frac_idx]

    # find that also for coarse_times
    max_frac_idx_coarse = findfirst(x -> x == max_frac_time, coarse_times)

    frustrated_flucs = all_frustrated_flucs[plot_seed]

    figure(figsize=(4, 3))
    title(plot_seed)
    ax = subplot(211)
    plot(padded_exact_times, shift_idx_to_center(max_frac_idx, total_fracs), "-k", lw=2.5)
    xlim(0., 1.)
    ylim(0, 10)
    ylabel("\$\\frac{|\\langle \\alpha|H_Z-H_X|0\\rangle|}{E_\\alpha - E_0}\$")
    ax.set_xticklabels([])
    axvline(minigap_locs[plot_seed], ls="--", c="k", alpha=0.5)
    # legend(frameon=false)

    ax = subplot(212)
    plot(padded_coarse_times,  shift_idx_to_center(max_frac_idx_coarse, frustrated_flucs .- mean_scaled_flucs[plot_seed]), label=plot_seed)
    plot(padded_coarse_times,  shift_idx_to_center(max_frac_idx_coarse, frustrated_flucs))
    xlim(0., 1.)
    ylim(0, )
    xlabel("\$s - s_*\$")
    legend(frameon=false)

    tight_layout()
    # savefig(("test_plots/seed_" * plot_seed * ".pdf"))
end

In [ ]:
num_instances = 10
# for plot_seed in ordered_seeds
for (plot_seed, _) in seeds_and_max_fracs[1:num_instances]
    # plot_seed = ordered_seeds[20]
    print(plot_seed, "\t")
    frac_1, frac_2, frac_3, frac_4 = all_adiabatic_fracs[plot_seed]
    frustrated_flucs = all_frustrated_flucs[plot_seed];

    figure(figsize=(4, 3))
    title(plot_seed)
    ax = subplot(211)
    plot(exact_times, frac_1, label="\$\\alpha = 0\$")
    plot(exact_times, frac_2, label="\$\\alpha = 1\$", lw=1.5)
    plot(exact_times, frac_3, label="\$\\alpha = 2\$", lw=2)
    plot(exact_times, frac_4, label="\$\\alpha = 3\$", lw=2.5)
    plot(exact_times, sum(all_adiabatic_fracs[plot_seed]), "-k", lw=2.5)
    xlim(0., 1.)
    ylim(0, 10)
    ylabel("\$\\frac{|\\langle \\alpha|H_Z-H_X|0\\rangle|}{E_\\alpha - E_0}\$")
    ax.set_xticklabels([])
    axvline(minigap_locs[plot_seed], ls="--", c="k", alpha=0.5)
    # legend(frameon=false)

    ax = subplot(212)
    plot(coarse_times,  frustrated_flucs .- mean_scaled_flucs[plot_seed], label=plot_seed)
    plot(coarse_times,  frustrated_flucs)
    xlim(0., 1.)
    ylim(0, )
    xlabel("\$s\$")
    legend(frameon=false)

    tight_layout()
    # savefig(("test_plots/seed_" * plot_seed * ".pdf"))
end